# Imports

In [32]:
import os
    
# Main libraries that we will use in this kernel
import datetime
import numpy as np
import pandas as pd

# # garbage collector: free some memory is needed
import gc
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# statistical package and some useful functions to analyze our timeseries
from statsmodels.tsa.stattools import pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.tsa.stattools as stattools

import time

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings("ignore")

# Data Loading and Preprocessing

In [33]:
PATH_DATA = "../input/nuclio10-dsc-1121"

In [34]:
df = pd.read_csv(os.path.join(PATH_DATA, "sales_train_merged.csv"), index_col = 0)

In [35]:
df["date"] = pd.to_datetime(df["date"], format = "%Y-%m-%d")

In [36]:
df.head()

,date,city,shop_id,shop_name,item_category_id,item_category_name,item_id,item_name,unique_id,item_price,item_cnt_day
0,2013-01-03,Moscú,25,"Moscú SEC ""Atrium""",55,Música - CD de la producción local,2574,Del Rey Lana Nació para morir la edición de Pa...,25-2574,399.0,2.0
1,2013-01-05,Moscú,25,"Moscú SEC ""Atrium""",55,Música - CD de la producción local,2574,Del Rey Lana Nació para morir la edición de Pa...,25-2574,399.0,1.0
2,2013-01-07,Moscú,25,"Moscú SEC ""Atrium""",55,Música - CD de la producción local,2574,Del Rey Lana Nació para morir la edición de Pa...,25-2574,399.0,1.0
3,2013-01-08,Moscú,25,"Moscú SEC ""Atrium""",55,Música - CD de la producción local,2574,Del Rey Lana Nació para morir la edición de Pa...,25-2574,399.0,2.0
4,2013-01-10,Moscú,25,"Moscú SEC ""Atrium""",55,Música - CD de la producción local,2574,Del Rey Lana Nació para morir la edición de Pa...,25-2574,399.0,1.0


# ¿Cuáles han sido las ventas totales en la ciudad de Yaroslavl durante el año 2014?

In [37]:
city_y_sales = df.set_index("date").groupby(["city"]).resample("Y")["item_cnt_day"].sum().to_frame().reset_index()

In [38]:
city_y_sales[city_y_sales["city"] == "Yaroslavl"] # correcta 1.281

,city,date,item_cnt_day
71,Yaroslavl,2013-12-31,1100.0
72,Yaroslavl,2014-12-31,1281.0
73,Yaroslavl,2015-12-31,656.0


# La tercera ciudad con más ventas (durante todo el periodo) es:

In [39]:
df.groupby("city")["item_cnt_day"].sum().sort_values(ascending = False).to_frame().head(3) # correcta Yakutsk

,item_cnt_day
city,
Moscú,112899.0
Tienda Online,26338.0
Yakutsk,19016.0


# El item_name mas caro del año 2013 es:

In [40]:
df["year"] = df["date"].dt.year

max_price_gb = df.groupby(["year"])["item_price"].max()
max_price_2013 = max_price_gb.loc[2013]

df[(df["year"] == 2013) & (df["item_price"] == max_price_2013)]

,date,city,shop_id,shop_name,item_category_id,item_category_name,item_id,item_name,unique_id,item_price,item_cnt_day,year
15432,2013-09-17,Tienda Online,12,La tienda en línea Emergencias,9,Entrega de bienes,11365,Entrega (EMS),12-11365,59200.0,1.0,2013


# La media móvil de 3 meses para la tienda centro de Samara "Parkhouse" a fecha 2013-03-31 (incluyendo el mes) es:

Debemos hacer primero el resample por Mes y por Tienda (porque las ventas vienen por día).<br>
Una vez que tenemos las ventas mensuales de cada ciudad, debemos volver a hacer un groupby para sacar la media móvil mensual de tres meses.<br>
De allí que hacemos 2 groupy en la siguiente celda.

In [41]:
rolling_mean_3m_city = df.set_index("date").groupby(["shop_name"]).resample("M")["item_cnt_day"].sum().to_frame().reset_index().set_index("date").groupby("shop_name")["item_cnt_day"].rolling(window = 3).mean().to_frame().reset_index()

In [42]:
rolling_mean_3m_city[rolling_mean_3m_city["shop_name"] == 'centro de Samara "Parkhouse"']

,shop_name,date,item_cnt_day
1215,"centro de Samara ""Parkhouse""",2013-01-31,NaN
1216,"centro de Samara ""Parkhouse""",2013-02-28,NaN
1217,"centro de Samara ""Parkhouse""",2013-03-31,23.666667
1218,"centro de Samara ""Parkhouse""",2013-04-30,19.000000
1219,"centro de Samara ""Parkhouse""",2013-05-31,43.333333
1220,"centro de Samara ""Parkhouse""",2013-06-30,86.666667
1221,"centro de Samara ""Parkhouse""",2013-07-31,115.000000
1222,"centro de Samara ""Parkhouse""",2013-08-31,106.333333
1223,"centro de Samara ""Parkhouse""",2013-09-30,88.666667
1224,"centro de Samara ""Parkhouse""",2013-10-31,79.333333


# La segunda categoría más vendida durante el año 2014 ha sido:

In [43]:
df[df["year"] == 2014].groupby("item_category_name")["item_cnt_day"].sum().sort_values(ascending = False).head(3)

item_category_name
Regalos - Bolsas, álbumes, Mats d / ratón    65929.0
Entrega de bienes                             5952.0
Las tarjetas de pago - PSN                    5709.0
Name: item_cnt_day, dtype: float64

# El día con más ventas de todo el periodo corresponde a:

In [44]:
daily_sales = df.groupby("date")["item_cnt_day"].sum()
daily_sales[daily_sales == daily_sales.max()]

date
2015-05-19    1524.0
Name: item_cnt_day, dtype: float64

# La primera media móvil de 7 días (después de hacer dropna) corresponde a:

Nos pasa algo parecido que en la pregunta de Samara "Parkhouse".

Al ser las ventas por día pero en diferentes tiendas y artículos, primero debemos hacer la suma para posteriormente aplicar la media móvile de 7 días.

In [45]:
df.set_index("date").resample("D")["item_cnt_day"].sum().rolling(window = 7).mean().dropna()

date
2013-01-07     71.714286
2013-01-08     79.428571
2013-01-09     74.142857
2013-01-10     67.571429
2013-01-11     68.428571
                 ...    
2015-09-26    200.142857
2015-09-27    202.857143
2015-09-28    205.714286
2015-09-29    215.428571
2015-09-30    219.000000
Freq: D, Name: item_cnt_day, Length: 997, dtype: float64

# El item_id que menos días distintos se ha vendido es:

In [46]:
df.pivot_table(
    index = "date",
    columns = "item_id",
    values = "item_cnt_day",
    aggfunc = sum
).isnull().sum().sort_values()

item_id
5822       9
20949    113
17717    117
7856     118
2308     138
        ... 
16790    831
3007     832
6429     833
16287    836
15256    838
Length: 100, dtype: int64

# La ciudad que más tiendas tiene es:

In [47]:
df[["city", "shop_id"]].drop_duplicates().groupby("city").size().sort_values()

city
Balashikha         1
Volzhski           1
Vlogda             1
Tomsk              1
Surgut             1
San Petersburgo    1
Yaroslavl          1
Posad              1
Omsk               1
Zhukovsky          1
Kursk              1
Krasnoyarsk        1
Kolomna            1
Kazan              1
Kaluga             1
Checos             1
Novosibirsk        1
Nizhny Novgorod    2
Samara             2
Tienda Online      2
Tyumen             2
Ufa                2
Voronezh           2
Yakutsk            2
Rostov Na Donu     2
Moscú              7
dtype: int64

# El item_category_id que más se vende en la ciudad de Surgut es:

In [48]:
df[
    (df["city"] == "Surgut")
].groupby("item_category_id")["item_cnt_day"].sum().sort_values()

item_category_id
30     205.0
23     281.0
35     296.0
79     369.0
19     447.0
71    4365.0
Name: item_cnt_day, dtype: float64